# Hyperparameter Optimization & Neural Architecture Search

In [1]:
# Set the correct search path for importing packages
import sys
sys.path.insert(1, '../PINNLearning')
# Surpress unnecessary output from tensorflow package
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import necessary functions/ packages
from training import learning_rate_schedule, oneD_loss
from data import gen_data, set_boundaries
from models import create_model
import keras_tuner as kt
import tensorflow as tf
import keras

In [2]:
# Set the boundary conditions
x_bc, y_bc = set_boundaries([[0.0], [1.0]], [[1.0], [0.0]])

# Create the training data
x_train = gen_data(0.0, 1.0, 100)

# Pad the bc values and concatenate everthing needed for the loss function into 1 tensor
# as the keras tuner expects the input in the form of: (x_train, y_train)
x_bc_padded = tf.pad(x_bc, tf.constant([[0, len(x_train) - 2], [0, 0]]))
y_bc_padded = tf.pad(y_bc, tf.constant([[0, len(x_train) - 2], [0, 0]]))
y_train = tf.concat([x_train, x_bc_padded, y_bc_padded], axis=1)

In [3]:
# Description of all hyper parameters that need to be searched
def search_space(hp):
    # defining the learning rate schedule with its searchable parameters
    lr_schedule = learning_rate_schedule(
        hp.Float('init', min_value=0.5e-3, max_value=5e-3, step=0.1e-3),
        hp.Int('steps', min_value=100, max_value=2000),
        hp.Float('rate', min_value=0.1, max_value=0.99, step=0.01)
    )

    # create the model with searchable regularizer, #layers and #weights per layer
    l2_penalty = hp.Float('l2_penalty', min_value=0.005, max_value=0.5, step=1e-3)
    num_layers = hp.Int('#layers', min_value=2, max_value=10)
    weigts = []
    for i in range(num_layers):
        weigts.append(hp.Int(f'#weights layer {i + 1}', min_value=2, max_value=50))
    model = create_model(num_layers, weigts, l2=l2_penalty)

    # closure function to convert the expected input format (y_true, y_pred) for 
    # the loss into a format that can be used in the previously implemented function 
    def custom_loss(y_true, y_pred):
        x_train = y_true[:, 0]
        xbc = y_true[:, 1][0:2]
        ybc = y_true[:, 2][0:2]
        return oneD_loss(model, x_train, xbc, ybc)

    # finish the model by assigning the optimizer and the loss
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=lr_schedule),
        loss=custom_loss)

    return model

Kein BOHB vorhanden

--> entnehme direkt aus hyperband

In [4]:
# Defining the tuner as BayesianOptimization with the search space,
# the minimizing target, the number of iterations of the
# and the folder in which the collected data is saved
tuner = kt.BayesianOptimization(
    search_space,
    objective='val_loss',
    max_trials=50,
    directory='../data/HPO_results')

Reloading Tuner from ../data/HPO_results/untitled_project/tuner0.json


In [5]:
# Creating callbacks to reduce uneccesarry training time and inculde 
# output for the tensorboard visuallization below
tensorboard_cb = keras.callbacks.TensorBoard(log_dir='../data/HPO_results')
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', baseline=1e-3, patience=0)

# Search through the space using the tuner
tuner.search(
    x_train,
    y_train,
    epochs=60,  # Train every iteration with this number of epochs
    validation_split=0.2,  # Use random 20% of the data as validation
    callbacks=[tensorboard_cb, stop_early])


So the keras tuner, will create automatically the "maximal" space and tests points thought it. If eg. the number of layers it returns to me is 2, but I have maximally 5, I can just ignore the values for the other ones.

In [7]:
# Get and print the best combinations of the HPs
best_hps = tuner.get_best_hyperparameters()[0].values
for hp in best_hps:
    print(f"{hp}: {best_hps.get(hp)}")

    # Stop printing if the max number of layers is lower than
    # the current layer for the number of weights 
    if hp[-1].isdigit() and int(hp[-1]) >= best_hps.get('#layers'):
        break

# Load the TensorBoard interactive interface for more information
# FYI: Might not show directly, just re-run this cell
%reload_ext tensorboard
%tensorboard --logdir ../data/HPO_results

init: 0.0049
steps: 1877
rate: 0.58
l2_penalty: 0.434
#layers: 2
#weights layer 1: 32
#weights layer 2: 31


Reusing TensorBoard on port 6006 (pid 24187), started 0:00:04 ago. (Use '!kill 24187' to kill it.)